In [20]:
%matplotlib widget
import torch
import torch.nn as nn
import pickle
import time
import copy
import os
from queue import LifoQueue
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import davies_bouldin_score
from scipy.stats import kurtosis
import matplotlib.pyplot as plt
import numpy as np
import sys
from sklearn.tree import DecisionTreeClassifier
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from tqdm.notebook import tqdm
from sklearn.metrics import pairwise_distances
import network.cpc
from network.cpc import CDCK2
from utils.MatplotlibUtils import reduce_dims_and_plot
from utils.ClassificationUtiols import onehot_coding
from soft_decision_tree.sdt_model import SDT


# IPython.OutputArea.auto_scroll_threshold = 9999;

# Load the model and the dataset

In [2]:
model_path = r'/home/eitan.k/EntangledExplainableClustering/knn_loss_batch_512_k_32_min_max_normalization/models/epoch_100.pt'
dataset_path = r'/home/eitan.k/EntangledExplainableClustering/knn_loss_batch_512_k_32_min_max_normalization/data/test_data.file'

print(f"Load the model from: {model_path}")
model = torch.load(model_path, map_location='cpu')

with open(dataset_path, 'rb') as fp:
    dataset = pickle.load(fp)

Load the model from: /home/eitan.k/EntangledExplainableClustering/knn_loss_batch_512_k_32_min_max_normalization/models/epoch_100.pt


In [3]:
batch_size = 500
loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Extract representations

In [4]:
projections = torch.tensor([])
samples = torch.tensor([])
device = 'cuda'
model = model.to(device).eval()

with torch.no_grad():
    bar = tqdm(total=len(loader.dataset))
    for batch in loader:
        hidden = CDCK2.init_hidden(len(batch))
        batch = batch.to(device)
        hidden = hidden.to(device)

        y = model.predict(batch, hidden).detach().cpu()
        projections = torch.cat([projections, y.detach().cpu()])
        samples = torch.cat([samples, batch.detach().cpu()])
        bar.update(y.shape[0])

  0%|          | 0/25367 [00:00<?, ?it/s]

# Fit GMM and calculate indices

In [5]:
scores = []
best_score = float('inf')
clusters = None
range_ = list(range(5, 20))
for k in tqdm(range_):
    y = GaussianMixture(n_components=k).fit_predict(projections)
    cur_score = davies_bouldin_score(projections, y)
    scores.append(cur_score)
    
    if cur_score < best_score:
        best_score = cur_score
        clusters = y

  0%|          | 0/15 [00:00<?, ?it/s]

In [6]:
plt.figure()
plt.xlabel('Number of clusters')
plt.ylabel('DB Score')
plt.plot(range_, scores)
best_k = range_[np.argmin(scores)]
plt.axvline(best_k, color='r')
plt.show()

labels = set(clusters)
print(labels)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}


In [7]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Visualize with T-SNE

In [8]:
perplexity = 200

p = reduce_dims_and_plot(projections,
                         y=clusters,
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [9]:
# unnormalized_samples = samples.clone()

# for col, sensor in enumerate(tqdm(dataset.dataset.all_signals)):
#     denormalizer = dataset.dataset.get_denormalization_for_sensor(sensor)
#     unnormalized_samples[:, col, :] = denormalizer(unnormalized_samples[:, col, :])

sampled = samples[..., range(0, samples.shape[-1], 200)]

samples_f = sampled.flatten(1)
tree_dataset = list(zip(samples_f, clusters))
batch_size = 2000
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [10]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [11]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [18]:
lr = 1e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 500
output_dim = len(set(clusters))
log_interval = 1
tree_depth = 10

In [13]:
tree = SDT(input_dim=samples_f.shape[1], output_dim=len(labels), depth=tree_depth, lamda=1e-3, use_cuda=True)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)
clf = DecisionTreeClassifier(max_depth=d).fit(samples_f, clusters)
tree.

In [14]:
losses = []
accs = []
sparsity = []

In [19]:
tree = tree.train()
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=5)


Average sparseness: 0.9723756906077345
layer 0: 0.9723756906077348
layer 1: 0.9723756906077348
layer 2: 0.9723756906077348
layer 3: 0.9723756906077348
layer 4: 0.9723756906077348
layer 5: 0.9723756906077348
layer 6: 0.9723756906077349
layer 7: 0.9723756906077345
layer 8: 0.9723756906077345
Epoch: 00 | Batch: 000 / 013 | Total loss: 1.324 | Reg loss: 0.037 | Tree loss: 1.324 | Accuracy: 0.513000 | 0.902 sec/iter
Epoch: 00 | Batch: 001 / 013 | Total loss: 1.252 | Reg loss: 0.037 | Tree loss: 1.252 | Accuracy: 0.547000 | 0.868 sec/iter
Epoch: 00 | Batch: 002 / 013 | Total loss: 1.229 | Reg loss: 0.037 | Tree loss: 1.229 | Accuracy: 0.560500 | 0.859 sec/iter
Epoch: 00 | Batch: 003 / 013 | Total loss: 1.213 | Reg loss: 0.037 | Tree loss: 1.213 | Accuracy: 0.555500 | 0.855 sec/iter
Epoch: 00 | Batch: 004 / 013 | Total loss: 1.183 | Reg loss: 0.037 | Tree loss: 1.183 | Accuracy: 0.564500 | 0.854 sec/iter
Epoch: 00 | Batch: 005 / 013 | Total loss: 1.136 | Reg loss: 0.037 | Tree loss: 1.136 | A

Epoch: 04 | Batch: 003 / 013 | Total loss: 1.226 | Reg loss: 0.037 | Tree loss: 1.226 | Accuracy: 0.552000 | 0.858 sec/iter
Epoch: 04 | Batch: 004 / 013 | Total loss: 1.184 | Reg loss: 0.037 | Tree loss: 1.184 | Accuracy: 0.585000 | 0.858 sec/iter
Epoch: 04 | Batch: 005 / 013 | Total loss: 1.150 | Reg loss: 0.037 | Tree loss: 1.150 | Accuracy: 0.588000 | 0.858 sec/iter
Epoch: 04 | Batch: 006 / 013 | Total loss: 1.133 | Reg loss: 0.038 | Tree loss: 1.133 | Accuracy: 0.613500 | 0.858 sec/iter
Epoch: 04 | Batch: 007 / 013 | Total loss: 1.152 | Reg loss: 0.038 | Tree loss: 1.152 | Accuracy: 0.624500 | 0.858 sec/iter
Epoch: 04 | Batch: 008 / 013 | Total loss: 1.120 | Reg loss: 0.038 | Tree loss: 1.120 | Accuracy: 0.622500 | 0.858 sec/iter
Epoch: 04 | Batch: 009 / 013 | Total loss: 1.105 | Reg loss: 0.038 | Tree loss: 1.105 | Accuracy: 0.629500 | 0.858 sec/iter
Epoch: 04 | Batch: 010 / 013 | Total loss: 1.121 | Reg loss: 0.038 | Tree loss: 1.121 | Accuracy: 0.617000 | 0.858 sec/iter
Epoch: 0

Epoch: 08 | Batch: 008 / 013 | Total loss: 1.102 | Reg loss: 0.038 | Tree loss: 1.102 | Accuracy: 0.643500 | 0.857 sec/iter
Epoch: 08 | Batch: 009 / 013 | Total loss: 1.132 | Reg loss: 0.038 | Tree loss: 1.132 | Accuracy: 0.636500 | 0.857 sec/iter
Epoch: 08 | Batch: 010 / 013 | Total loss: 1.112 | Reg loss: 0.038 | Tree loss: 1.112 | Accuracy: 0.639000 | 0.857 sec/iter
Epoch: 08 | Batch: 011 / 013 | Total loss: 1.120 | Reg loss: 0.038 | Tree loss: 1.120 | Accuracy: 0.619000 | 0.857 sec/iter
Epoch: 08 | Batch: 012 / 013 | Total loss: 1.121 | Reg loss: 0.038 | Tree loss: 1.121 | Accuracy: 0.621068 | 0.856 sec/iter
Average sparseness: 0.9723756906077345
layer 0: 0.9723756906077348
layer 1: 0.9723756906077348
layer 2: 0.9723756906077348
layer 3: 0.9723756906077348
layer 4: 0.9723756906077348
layer 5: 0.9723756906077348
layer 6: 0.9723756906077349
layer 7: 0.9723756906077345
layer 8: 0.9723756906077345
Epoch: 09 | Batch: 000 / 013 | Total loss: 1.305 | Reg loss: 0.037 | Tree loss: 1.305 | A

Epoch: 13 | Batch: 000 / 013 | Total loss: 1.303 | Reg loss: 0.037 | Tree loss: 1.303 | Accuracy: 0.521000 | 0.857 sec/iter
Epoch: 13 | Batch: 001 / 013 | Total loss: 1.248 | Reg loss: 0.037 | Tree loss: 1.248 | Accuracy: 0.538500 | 0.857 sec/iter
Epoch: 13 | Batch: 002 / 013 | Total loss: 1.232 | Reg loss: 0.037 | Tree loss: 1.232 | Accuracy: 0.551000 | 0.857 sec/iter
Epoch: 13 | Batch: 003 / 013 | Total loss: 1.197 | Reg loss: 0.037 | Tree loss: 1.197 | Accuracy: 0.560000 | 0.857 sec/iter
Epoch: 13 | Batch: 004 / 013 | Total loss: 1.179 | Reg loss: 0.038 | Tree loss: 1.179 | Accuracy: 0.570000 | 0.857 sec/iter
Epoch: 13 | Batch: 005 / 013 | Total loss: 1.167 | Reg loss: 0.038 | Tree loss: 1.167 | Accuracy: 0.558500 | 0.857 sec/iter
Epoch: 13 | Batch: 006 / 013 | Total loss: 1.154 | Reg loss: 0.038 | Tree loss: 1.154 | Accuracy: 0.571000 | 0.857 sec/iter
Epoch: 13 | Batch: 007 / 013 | Total loss: 1.122 | Reg loss: 0.038 | Tree loss: 1.122 | Accuracy: 0.610500 | 0.857 sec/iter
Epoch: 1

Epoch: 17 | Batch: 005 / 013 | Total loss: 1.173 | Reg loss: 0.038 | Tree loss: 1.173 | Accuracy: 0.570000 | 0.857 sec/iter
Epoch: 17 | Batch: 006 / 013 | Total loss: 1.146 | Reg loss: 0.038 | Tree loss: 1.146 | Accuracy: 0.579500 | 0.857 sec/iter
Epoch: 17 | Batch: 007 / 013 | Total loss: 1.135 | Reg loss: 0.038 | Tree loss: 1.135 | Accuracy: 0.599000 | 0.857 sec/iter
Epoch: 17 | Batch: 008 / 013 | Total loss: 1.112 | Reg loss: 0.038 | Tree loss: 1.112 | Accuracy: 0.632000 | 0.857 sec/iter
Epoch: 17 | Batch: 009 / 013 | Total loss: 1.119 | Reg loss: 0.038 | Tree loss: 1.119 | Accuracy: 0.627500 | 0.857 sec/iter
Epoch: 17 | Batch: 010 / 013 | Total loss: 1.113 | Reg loss: 0.038 | Tree loss: 1.113 | Accuracy: 0.625500 | 0.857 sec/iter
Epoch: 17 | Batch: 011 / 013 | Total loss: 1.110 | Reg loss: 0.038 | Tree loss: 1.110 | Accuracy: 0.636500 | 0.857 sec/iter
Epoch: 17 | Batch: 012 / 013 | Total loss: 1.106 | Reg loss: 0.038 | Tree loss: 1.106 | Accuracy: 0.629846 | 0.856 sec/iter
Average 

...

Epoch: 455 | Batch: 000 / 013 | Total loss: 1.282 | Reg loss: 0.038 | Tree loss: 1.282 | Accuracy: 0.545000 | 0.857 sec/iter
Epoch: 455 | Batch: 001 / 013 | Total loss: 1.260 | Reg loss: 0.038 | Tree loss: 1.260 | Accuracy: 0.539500 | 0.857 sec/iter
Epoch: 455 | Batch: 002 / 013 | Total loss: 1.252 | Reg loss: 0.038 | Tree loss: 1.252 | Accuracy: 0.540500 | 0.857 sec/iter
Epoch: 455 | Batch: 003 / 013 | Total loss: 1.192 | Reg loss: 0.038 | Tree loss: 1.192 | Accuracy: 0.570000 | 0.857 sec/iter
Epoch: 455 | Batch: 004 / 013 | Total loss: 1.176 | Reg loss: 0.038 | Tree loss: 1.176 | Accuracy: 0.583000 | 0.857 sec/iter
Epoch: 455 | Batch: 005 / 013 | Total loss: 1.160 | Reg loss: 0.038 | Tree loss: 1.160 | Accuracy: 0.568000 | 0.857 sec/iter
Epoch: 455 | Batch: 006 / 013 | Total loss: 1.139 | Reg loss: 0.038 | Tree loss: 1.139 | Accuracy: 0.595500 | 0.857 sec/iter
Epoch: 455 | Batch: 007 / 013 | Total loss: 1.134 | Reg loss: 0.038 | Tree loss: 1.134 | Accuracy: 0.593000 | 0.857 sec/iter


Epoch: 459 | Batch: 005 / 013 | Total loss: 1.162 | Reg loss: 0.038 | Tree loss: 1.162 | Accuracy: 0.587500 | 0.857 sec/iter
Epoch: 459 | Batch: 006 / 013 | Total loss: 1.156 | Reg loss: 0.038 | Tree loss: 1.156 | Accuracy: 0.597500 | 0.857 sec/iter
Epoch: 459 | Batch: 007 / 013 | Total loss: 1.120 | Reg loss: 0.038 | Tree loss: 1.120 | Accuracy: 0.606000 | 0.857 sec/iter
Epoch: 459 | Batch: 008 / 013 | Total loss: 1.106 | Reg loss: 0.038 | Tree loss: 1.106 | Accuracy: 0.602000 | 0.857 sec/iter
Epoch: 459 | Batch: 009 / 013 | Total loss: 1.100 | Reg loss: 0.038 | Tree loss: 1.100 | Accuracy: 0.630000 | 0.857 sec/iter
Epoch: 459 | Batch: 010 / 013 | Total loss: 1.119 | Reg loss: 0.038 | Tree loss: 1.119 | Accuracy: 0.598500 | 0.857 sec/iter
Epoch: 459 | Batch: 011 / 013 | Total loss: 1.108 | Reg loss: 0.038 | Tree loss: 1.108 | Accuracy: 0.613500 | 0.857 sec/iter
Epoch: 459 | Batch: 012 / 013 | Total loss: 1.102 | Reg loss: 0.038 | Tree loss: 1.102 | Accuracy: 0.616679 | 0.857 sec/iter


Epoch: 463 | Batch: 010 / 013 | Total loss: 1.097 | Reg loss: 0.038 | Tree loss: 1.097 | Accuracy: 0.627000 | 0.857 sec/iter
Epoch: 463 | Batch: 011 / 013 | Total loss: 1.087 | Reg loss: 0.038 | Tree loss: 1.087 | Accuracy: 0.644500 | 0.857 sec/iter
Epoch: 463 | Batch: 012 / 013 | Total loss: 1.119 | Reg loss: 0.038 | Tree loss: 1.119 | Accuracy: 0.620337 | 0.857 sec/iter
Average sparseness: 0.9723756906077345
layer 0: 0.9723756906077348
layer 1: 0.9723756906077348
layer 2: 0.9723756906077348
layer 3: 0.9723756906077348
layer 4: 0.9723756906077348
layer 5: 0.9723756906077348
layer 6: 0.9723756906077349
layer 7: 0.9723756906077345
layer 8: 0.9723756906077345
Epoch: 464 | Batch: 000 / 013 | Total loss: 1.300 | Reg loss: 0.038 | Tree loss: 1.300 | Accuracy: 0.514500 | 0.857 sec/iter
Epoch: 464 | Batch: 001 / 013 | Total loss: 1.269 | Reg loss: 0.038 | Tree loss: 1.269 | Accuracy: 0.532000 | 0.857 sec/iter
Epoch: 464 | Batch: 002 / 013 | Total loss: 1.246 | Reg loss: 0.038 | Tree loss: 1.2

Epoch: 468 | Batch: 000 / 013 | Total loss: 1.297 | Reg loss: 0.038 | Tree loss: 1.297 | Accuracy: 0.520500 | 0.857 sec/iter
Epoch: 468 | Batch: 001 / 013 | Total loss: 1.259 | Reg loss: 0.038 | Tree loss: 1.259 | Accuracy: 0.536500 | 0.857 sec/iter
Epoch: 468 | Batch: 002 / 013 | Total loss: 1.212 | Reg loss: 0.038 | Tree loss: 1.212 | Accuracy: 0.555000 | 0.857 sec/iter
Epoch: 468 | Batch: 003 / 013 | Total loss: 1.193 | Reg loss: 0.038 | Tree loss: 1.193 | Accuracy: 0.555500 | 0.857 sec/iter
Epoch: 468 | Batch: 004 / 013 | Total loss: 1.163 | Reg loss: 0.038 | Tree loss: 1.163 | Accuracy: 0.575000 | 0.857 sec/iter
Epoch: 468 | Batch: 005 / 013 | Total loss: 1.142 | Reg loss: 0.038 | Tree loss: 1.142 | Accuracy: 0.591000 | 0.857 sec/iter
Epoch: 468 | Batch: 006 / 013 | Total loss: 1.130 | Reg loss: 0.038 | Tree loss: 1.130 | Accuracy: 0.621500 | 0.857 sec/iter
Epoch: 468 | Batch: 007 / 013 | Total loss: 1.137 | Reg loss: 0.038 | Tree loss: 1.137 | Accuracy: 0.642000 | 0.857 sec/iter


KeyboardInterrupt: 

In [32]:
scores = []
for d in range(5, 26):
    clf = DecisionTreeClassifier(max_depth=d).fit(samples_f, clusters)
    s = clf.score(samples_f, clusters)
    scores.append(s)
    print(f"depth: {d} | score: {s}")

depth: 5 | score: 0.8127882682224938
depth: 6 | score: 0.8932865533961446
depth: 7 | score: 0.9375566681121141
depth: 8 | score: 0.9539165056963772
depth: 9 | score: 0.9731540978436551
depth: 10 | score: 0.9798162967635117
depth: 11 | score: 0.9896716206094532
depth: 12 | score: 0.9942839121693539
depth: 13 | score: 0.9963338195293098
depth: 14 | score: 0.9973587732092877
depth: 15 | score: 0.9979106713446604
depth: 16 | score: 0.998147199116963
depth: 17 | score: 0.9986596759569519
depth: 18 | score: 0.9987779398431033
depth: 19 | score: 0.9992115740923246
depth: 20 | score: 0.9994086805692435
depth: 21 | score: 0.9995663657507786
depth: 22 | score: 0.9996846296369298
depth: 23 | score: 0.9997634722276973
depth: 24 | score: 0.9999211574092325
depth: 25 | score: 1.0


In [30]:
print(clf.get_depth())
print(clf.get_n_leaves())
print(clf.score(samples_f, clusters))

6
54
0.8932865533961446


In [48]:
n = 0
clf.tree_.feature[n], clf.tree_.feature[n]

(149, 149)

In [ ]:
# prune_tree(tree, factor=1.5)
correct = 0
tree = tree.eval()
with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
            data, target = data.to(device), target.to(device)
            output = tree.forward(data)
            pred = output.data.max(1)[1]
            correct += pred.eq(target.view(-1).data).sum()

print(f"Accuracy: {correct / len(tree_loader.dataset)}")

In [ ]:
print(f"Sparseness: {sparseness(tree.inner_nodes.weight)}")
layer = 0
sps = []
for i in range(tree.inner_nodes.weight.shape[0]):
    cur_layer = np.floor(np.log2(i+1))
    if cur_layer != layer:
        print(f"layer {layer}: {np.mean(sps)}")
        sps = []
        layer = cur_layer
    
    x_ = tree.inner_nodes.weight[i, :]
    sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
    sps.append(sp)

In [ ]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

In [ ]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

## Prune the weights

In [ ]:
weights = tree.inner_nodes.weight.cpu().detach().numpy()
for i in range(0, weights.shape[0], 20):
    plt.figure()
    weights_layer = weights[i, :]
    plt.hist(weights_layer, bins=500)
    weights_std = np.std(weights_layer)
    weights_mean = np.mean(weights_layer)
    plt.axvline(weights_mean + weights_std, color='r')
    plt.axvline(weights_mean - weights_std, color='r')
    plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}\n Kurtosis: {kurtosis(weights_layer)}")
    plt.yscale("log")
    plt.show()

## Verify that the accuracy didn't change too much

In [ ]:
correct = 0
tree_copy = tree_copy.eval()
with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
            data, target = data.to(device), target.to(device)
            output = tree_copy.forward(data)
            pred = output.data.max(1)[1]
            correct += pred.eq(target.view(-1).data).sum()

print(f"Accuracy: {correct / len(tree_loader.dataset)}")


In [ ]:
weights = tree_copy.inner_nodes.weight.cpu().detach().numpy()
for i in range(0, weights.shape[0], 20):
    plt.figure()
    weights_layer = weights[i, :]
    plt.hist(weights_layer, bins=500)
    weights_std = np.std(weights_layer)
    weights_mean = np.mean(weights_layer)
    plt.axvline(weights_mean + weights_std, color='r')
    plt.axvline(weights_mean - weights_std, color='r')
    plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
    plt.yscale("log")
    plt.show()

# Tree Visualization

In [ ]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

# Extract Rules

# Accumulate samples in the leaves

In [ ]:
method = 'greedy'

In [ ]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

# Tighten boundaries

In [ ]:
signal_names = dataset.dataset.all_signals
normalizers = torch.tensor([])
for signal_name in signal_names:
    attr_names += [f"T{i}.{signal_name}" for i in range(sampled.shape[-1])]
    normalizers = torch.cat([dataset.dataset.sensor_maxs[signal_name] for _ in range(sampled.shape[-1])])
    

# print(attr_names)
leaves = root.get_leaves()
for leaf in leaves:
    leaf.reset_path()
    leaf.update_path_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    for cond in conds:
        cond.weights = cond.weights / normalizers
    print(f"============== Rule {rule_counter} ==============")

In [ ]:

stack = LifoQueue()
edge_stack = LifoQueue()
stack.put(root)
rule_counter = 0
root.reset()
while not stack.empty():
    node = stack.get()
    if node.is_leaf():
        print(f"============== Rule {rule_counter} ==============")
        for stack_node, cond in zip(stack.queue, edge_stack.queue[1:]):
            print(repr(stack_node.get_condition(attr_names)) + cond)
            print()
        
        rule_counter += 1
        edge_stack.get()
        continue
          
    if node.left is not None and not node.left.visited:
        stack.put(node)
        stack.put(node.left)
        node.left.visited = True
        edge_stack.put(' < 0')
        continue
        
    if node.right is not None and not node.right.visited:
        stack.put(node)
        stack.put(node.right)
        node.right.visited = True
        edge_stack.put(' > 0')
        continue
        
    if node is not root:
        edge_stack.get()